In [103]:
import pandas
import pprint
import pandas as pd
import requests
import json
import numpy as np

In [104]:
yearsToCheck = list((np.arange(2000, 2020, 1)).astype(str))
developedCountries = ['Australia','Japan','Sweden','Switzerland','USA']
developingCountries = ['Brazil','Egypt','Greece','India','South Africa']
underdevelopedCountries = ['Afghanistan','Bangladesh','Cambodia','Ethiopia','Senegal']

In [105]:
populationCsvPath = 'raw_information/pop.csv'
readPopCsv = pd.read_csv(populationCsvPath)
populationDataFrame = (pd.DataFrame(readPopCsv).set_index('country'))[yearsToCheck]

developed_populationDataFrame = populationDataFrame.loc[developedCountries]
developing_populationDataFrame = populationDataFrame.loc[developingCountries]
underdeveloped_populationDataFrame = populationDataFrame.loc[underdevelopedCountries]

developed_populationDataFrame.head()
developing_populationDataFrame.head()
underdeveloped_populationDataFrame.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
country,,,,,,,,,,,,,,,,,,,,
Afghanistan,19.5M,19.7M,21M,22.6M,23.6M,24.4M,25.4M,25.9M,26.4M,27.4M,28.2M,29.2M,30.5M,31.5M,32.7M,33.8M,34.6M,35.6M,36.7M,37.8M
Bangladesh,129M,132M,134M,137M,139M,141M,143M,144M,145M,147M,148M,150M,152M,154M,156M,158M,160M,162M,164M,166M
Cambodia,12.1M,12.3M,12.6M,12.8M,13M,13.2M,13.5M,13.7M,13.9M,14.2M,14.4M,14.6M,14.8M,15M,15.2M,15.4M,15.6M,15.8M,16M,16.2M
Ethiopia,67M,69M,71.1M,73.2M,75.3M,77.5M,79.7M,82M,84.4M,86.8M,89.2M,91.8M,94.5M,97.1M,99.7M,102M,105M,108M,111M,114M
Senegal,9.7M,9.94M,10.2M,10.4M,10.7M,11M,11.3M,11.6M,11.9M,12.2M,12.5M,12.9M,13.2M,13.6M,14M,14.4M,14.8M,15.2M,15.6M,16M


In [106]:
gdpUSDPath = 'raw_information/total_gdp_us_inflation_adjusted.csv'
readGdpUSDCsv = pd.read_csv(gdpUSDPath)
GdpDataFrame = (pd.DataFrame(readGdpUSDCsv).set_index('country'))[yearsToCheck]

developed_GdpDataFrame = GdpDataFrame.loc[developedCountries]
developing_GdpDataFrame = GdpDataFrame.loc[developingCountries]
underdeveloped_GdpDataFrame = GdpDataFrame.loc[underdevelopedCountries]

developed_GdpDataFrame.to_csv('raw_information/sliced_data/gdpUSD/')
developing_GdpDataFrame.to_csv('raw_information/sliced_data/gdpUSD/')
underdeveloped_GdpDataFrame.to_csv('raw_information/sliced_data/gdpUSD/')

In [107]:
avgDailyIncPpPath = 'raw_information/mincpcap_cppp.csv'
readvgDailyIncCsv = pd.read_csv(avgDailyIncPpPath)
avgDailyIncPpDataFrame = (pd.DataFrame(readvgDailyIncCsv).set_index('country'))[yearsToCheck]

developed_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developedCountries]
developing_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[developingCountries]
underdeveloped_avgDailyIncPpDataFrame = avgDailyIncPpDataFrame.loc[underdevelopedCountries]

developed_avgDailyIncPpDataFrame.to_csv('raw_information/sliced_data/')
developing_avgDailyIncPpDataFrame.to_csv('raw_information/sliced_data/')
underdeveloped_avgDailyIncPpDataFrame.to_csv('raw_information/sliced_data/')

In [108]:
# This is in tonnes
co2PerCapPath = 'raw_information/co2_pcap_cons.csv'
readco2PerCapCsv = pd.read_csv(co2PerCapPath)
co2PerCapDataFrame = (pd.DataFrame(readco2PerCapCsv).set_index('country'))[yearsToCheck]

developed_co2PerCapDataFrame = co2PerCapDataFrame.loc[developedCountries]
developing_co2PerCapDataFrame = co2PerCapDataFrame.loc[developingCountries]
underdeveloped_co2PerCapDataFrame = co2PerCapDataFrame.loc[underdevelopedCountries]

developed_co2PerCapDataFrame.to_csv('raw_information/sliced_data/')
developing_co2PerCapDataFrame.to_csv('raw_information/sliced_data/')
underdeveloped_co2PerCapDataFrame.to_csv('raw_information/sliced_data/')

In [109]:
# This is in million tonnes
co2TotalEmissionsPath = 'raw_information/co2_cons.csv'
readco2TotalEmissionCsv = pd.read_csv(co2TotalEmissionsPath)
co2TotalEmissionsDataFrame = (pd.DataFrame(readco2TotalEmissionCsv).set_index('country'))[yearsToCheck]

developed_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developedCountries]
developing_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[developingCountries]
underdeveloped_co2TotalEmissionsDataFrame = co2TotalEmissionsDataFrame.loc[underdevelopedCountries]

developed_co2TotalEmissionsDataFrame.to_csv('raw_information/sliced_data/')
developing_co2TotalEmissionsDataFrame.to_csv('raw_information/sliced_data/')
underdeveloped_co2TotalEmissionsDataFrame.to_csv('raw_information/sliced_data/')